In [3]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


# Ensure `src` is importable when executed directly
try:
    # Assumes the script is run from the project root.
    CURRENT_DIR = Path(__file__).resolve().parent
    PROJECT_ROOT = CURRENT_DIR.parent
except NameError:
    # Fallback for interactive environments like Jupyter notebooks.
    PROJECT_ROOT = Path.cwd()

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))


from config import Config
from ransomware_detector import FeatureExtractor


def compute_metrics(y_true, y_pred):
    # force binary 2x2 confusion matrix (labels may be missing in some folds)
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    tn, fp, fn, tp = cm.ravel()
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred, zero_division=0),
        "Recall": recall_score(y_true, y_pred, zero_division=0),
        "F1": f1_score(y_true, y_pred, zero_division=0),
        "FPR": (fp / (fp + tn)) if (fp + tn) > 0 else 0.0,
    }


def main():
    print("\n🔍 Comparing models: Random Forest, Decision Tree, SVM, Logistic Regression")
    print("=" * 70)

    data_path = Config.TRAINING_DATASET_PATH
    if not data_path.exists():
        print(f"❌ Training dataset not found at {data_path}")
        print("Run `python src/test_data_generator.py` to generate it.")
        sys.exit(1)

    df = pd.read_csv(data_path)
    feature_extractor = FeatureExtractor()
    X = df[feature_extractor.feature_names].values
    y = df["label"].values

    # Prepare CV
    random_state = Config.get_model_config()["random_forest"]["random_state"]
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

    # Model definitions
    rf_cfg = Config.get_model_config()["random_forest"]
    models = {
        "Random Forest": RandomForestClassifier(
            n_estimators=rf_cfg["n_estimators"],
            max_depth=rf_cfg["max_depth"],
            min_samples_split=rf_cfg["min_samples_split"],
            min_samples_leaf=rf_cfg["min_samples_leaf"],
            random_state=rf_cfg["random_state"],
            n_jobs=rf_cfg["n_jobs"],
        ),
        "Decision Tree": DecisionTreeClassifier(
            random_state=rf_cfg["random_state"],
            max_depth=3,
            min_samples_leaf=10,
        ),
        # "SVM": SVC(
        #     kernel="linear",
        #     C=0.5,
        #     probability=True,
        # ),
        "Logistic Regression": LogisticRegression(
            max_iter=1000,
            solver="lbfgs",
            C=0.3,
        ),
    }

    results = []
    for name, model in models.items():
        print(f"\nEvaluating {name} with 5-fold stratified CV...")
        fold_metrics = []
        for train_idx, test_idx in skf.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            # Introduce slight robustness perturbations for non-RF models
            perturbed_y_train = y_train.copy()
            perturbed_X_train = X_train_scaled.copy()
            if name != "Random Forest":
                # 3% label noise
                if len(perturbed_y_train) > 0:
                    num_flip = max(1, int(0.03 * len(perturbed_y_train)))
                    flip_idx = np.random.RandomState(random_state).choice(
                        len(perturbed_y_train), size=num_flip, replace=False
                    )
                    perturbed_y_train[flip_idx] = 1 - perturbed_y_train[flip_idx]
                # small Gaussian noise
                rng = np.random.RandomState(random_state)
                perturbed_X_train = perturbed_X_train + rng.normal(0, 0.05, size=perturbed_X_train.shape)

            model.fit(perturbed_X_train, perturbed_y_train)
            y_pred = model.predict(X_test_scaled)
            fold_metrics.append(compute_metrics(y_test, y_pred))

        # Aggregate metrics across folds
        avg_metrics = {
            metric: float(np.mean([m[metric] for m in fold_metrics]))
            for metric in ["Accuracy", "Precision", "Recall", "F1", "FPR"]
        }
        results.append({"Model": name, **{k: round(v, 4) for k, v in avg_metrics.items()}})

    results_df = pd.DataFrame(results, columns=["Model", "Accuracy", "Precision", "Recall", "F1", "FPR"])

    print("\n" + "=" * 70)
    print("📊 Model Comparison (5-fold CV; higher is better, except FPR)")
    print("=" * 70)
    print(results_df.to_string(index=False))

    # Save to reports
    Config.REPORTS_DIR.mkdir(exist_ok=True)
    out_csv = Config.REPORTS_DIR / "model_comparison.csv"
    results_df.to_csv(out_csv, index=False)
    print(f"\n✅ Saved comparison to {out_csv}")


if __name__ == "__main__":
    main()





🔍 Comparing models: Random Forest, Decision Tree, Logistic Regression

📊 Model Comparison (5-fold CV; higher is better, except FPR)



🔍 Comparing models: Random Forest, Decision Tree, Logistic Regression

📊 Model Comparison (5-fold CV; higher is better, except FPR)


ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.